This notebook computes F-measure and draws Desicion boundary via knn for a clustering result given in a json file

In [ ]:
# minimal imports
import torch, yaml, os, json
import ricci_regularization
import matplotlib.pyplot as plt
import numpy as np
from ricci_regularization import RiemannianKmeansTools

In [ ]:
k_means_setup_number = 2
setting_name = "Euclidean" # "Euclidean" or "Riemannian"
#setting_name = "Riemannian"
knn_neighbours_count = 7

In [ ]:
Path_clustering_setup = f"../experiments/MNIST_Setting_3/K_means_setup_{k_means_setup_number}"
if setting_name == "Riemannian":
    results_file_name = "params.json" # Riemannian setting
elif setting_name == "Euclidean":
    results_file_name = "Euclidean_k_means_params.json" # Euclidean setting
# Read from JSON file
with open(Path_clustering_setup + f"/{results_file_name}", "r") as f:
    params = json.load(f)
# Print or use the params dictionary
#print(params)

In [ ]:
ground_truth_labels = torch.tensor(params['ground_truth_labels'])
labels_assigned_by_k_means = torch.tensor(params[f'{setting_name}_k_means_labels'])
encoded_points = torch.tensor(params['encoded_points'])

# F_measure

In [ ]:
f_measure = RiemannianKmeansTools.compute_f_measure(labels_assigned_by_k_means, ground_truth_labels)
print("F-measure", f_measure)

In [ ]:
open(f"{Path_clustering_setup}/f_measure_{setting_name}.txt", "w").write(str(f_measure))
print(f"f-measure saved in {Path_clustering_setup}/f_measure_{setting_name}.txt")


# Decision boundary

In [ ]:
# by ground truth labels
RiemannianKmeansTools.plot_knn_decision_boundary(encoded_points,
        labels_for_coloring=ground_truth_labels,
        neighbours_number=knn_neighbours_count,
        saving_folder=Path_clustering_setup, 
        file_saving_name=f"Decision_boundary_ground_truth_labels")

In [ ]:
# by the chosen clustering method
RiemannianKmeansTools.plot_knn_decision_boundary(encoded_points,
        labels_for_coloring=labels_assigned_by_k_means,
        neighbours_number=knn_neighbours_count,
        saving_folder=Path_clustering_setup, 
        file_saving_name=f"Decision_boundary_{setting_name}_k-means_labels")